# Text Extractor from PDF Files
## Provenance
* Created new 2025-05-17 at Bignor!

In [7]:
from pathlib import Path 
import pandas as pd
import re
import numpy as np
from functools import partial
from pprint import pprint as pp
from random import sample

import fitz  # PyMuPDF
import Levenshtein

import archivum.mendeley_port as arcm
import archivum.library as arcl
import archivum.utilities as arcu
import archivum.document as arcd
import archivum.crossref as arcc
import archivum.gui as arcg
import archivum.reference as arcr
from archivum.utilities import fGT

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2025-05-28 14:29:45 +01:00)


In [4]:
lib = arcl.Library('uber-library')
lib.stats()

,references,documents
objects,6118,6132
no children,167,156
children,5951,5976
1 child,5833,5883
2 children,108,86
3 children,10,4
4+ children,0,3


time: 813 ms (started: 2025-05-28 14:28:57 +01:00)


In [124]:
fGT(lib.ref_df.querex('top 10 recent title ~ /Risk Measure/'))

tag,type,year,author,title
Ghossoub2024b,article,2024,"Ghossoub, Mario and Zhu, Michael Boyuan and Chong, Wing Fung",Pareto-Optimal Peer-to-Peer Risk Sharing with Robust Distortion Risk Measures
Bernard2024,article,2024,"Bernard, Carole and Pesenti, Silvana M. and Vanduffel, Steven",Robust distortion risk measures
Ghossoub2024,article,2024,"Ghossoub, Mario and Ren, Qinghua and Wang, Ruodu",Counter-monotonic risk allocations and distortion risk measures
Ghossoub2024c,article,2024,"Ghossoub, Mario and Ren, Qinghua and Wang, Ruodu",Counter-monotonic Risk Sharing with Heterogeneous Distortion Risk Measures
Yong2024,article,2024,"Yong, Yaodi and Cheung, Ka Chun and Zhang, Yiying",Optimal reinsurance design under distortion risk measures and reinsurer's default risk with partial recovery
Shen2024a,article,2024,"Shen, Yi and {Van Oosten}, Zachary and Wang, Ruodu",Partially Law-Invariant Risk Measures
Shen2024,article,2024,"Shen, Yi and {Van Oosten}, Zachary and Wang, Ruodu",Partially Law-Invariant Risk Measures
Bielecki2024,article,2024,"Bielecki, Tomasz R. and Cialenco, Igor and Liu, Hao",Time consistency of dynamic risk measures and dynamic performance measures generated by distortion functions
Huang2024,article,2024,"Huang, Muqiao and Wang, Ruodu",Coherent risk measures and uniform integrability
Bernard2023,article,2023,"Bernard, Carole and Pesenti, Silvana M. and Vanduffel, Steven",Robust distortion risk measures


time: 31 ms (started: 2025-05-28 15:32:45 +01:00)


In [5]:
lib.stats_ref_fields()

,distinct,missing
tag,6118,0
type,10,0
author,4662,5
doi,2539,3431
journal,1225,2643
pages,2749,2233
title,5902,0
volume,454,3143
year,100,256
publisher,169,5579


time: 47 ms (started: 2025-05-28 14:29:06 +01:00)


In [186]:
ps2 = list(Path('\\users\\steve\\downloads').glob('*.pdf'))
ps2

[WindowsPath('/users/steve/downloads/1-s2.0-S016766872500037X-main.pdf'),
 WindowsPath('/users/steve/downloads/1-s2.0-S0167668725000599-main.pdf'),
 WindowsPath('/users/steve/downloads/1176996608.pdf'),
 WindowsPath('/users/steve/downloads/12-AAP857.pdf'),
 WindowsPath('/users/steve/downloads/2023-Dhaene-Robert-Cheung-Denuit.pdf'),
 WindowsPath('/users/steve/downloads/2305.17743v2.pdf'),
 WindowsPath('/users/steve/downloads/3_Obstfeld.pdf'),
 WindowsPath('/users/steve/downloads/9309211v2.pdf'),
 WindowsPath('/users/steve/downloads/CT 12 10738 - Legacy Filing Packet 08-14-2012 {AFM ap}.pdf'),
 WindowsPath('/users/steve/downloads/Explanatory Memorandum.pdf'),
 WindowsPath('/users/steve/downloads/Gaussian emergence - accompanying note.pdf'),
 WindowsPath('/users/steve/downloads/Gaussian emergence for stochastic reserve risk modelling.pdf'),
 WindowsPath('/users/steve/downloads/gross-zagier.pdf'),
 WindowsPath('/users/steve/downloads/NJ HO 2024 Filing.pdf'),
 WindowsPath('/users/steve/down

time: 0 ns (started: 2025-05-28 10:18:09 +01:00)


In [ ]:
text_dir_name='\\temp\\pdf-full-text'
extractor='pdftotext'
pDocument = partial(arcd.Document, text_dir_path=Path(text_dir_name), extractor=extractor)
ps = lib.doc_df.sample(200).path

ans = []
for p in ps:
    p = Path(p)
    d = pDocument(p)
    a = d.meta_data(lib)
    ans.append([p.name, a.author, a.author_ex, a.subject, a.title, a.raw])

df = pd.DataFrame(ans, columns=['name', 'author', 'author_ex', 'subject', 'title', 'raw'])
fGT(df)

# GUI

In [10]:
def edit_book_info():
    import tkinter as tk
    from tkinter import messagebox

    def on_done():
        nonlocal result
        result = {
            "Title": title_var.get(),
            "Author": author_var.get(),
            "Year": year_var.get()
        }
        root.destroy()

    def on_cancel():
        root.destroy()

    result = None
    root = tk.Tk()
    root.title("Edit Book")

    tk.Label(root, text="Title").grid(row=0, column=0, sticky="e")
    tk.Label(root, text="Author").grid(row=1, column=0, sticky="e")
    tk.Label(root, text="Year").grid(row=2, column=0, sticky="e")

    title_var = tk.StringVar(value="The Great Gatsby")
    author_var = tk.StringVar(value="F. Scott Fitzgerald")
    year_var = tk.StringVar(value="1925")

    tk.Entry(root, textvariable=title_var).grid(row=0, column=1)
    tk.Entry(root, textvariable=author_var).grid(row=1, column=1)
    tk.Entry(root, textvariable=year_var).grid(row=2, column=1)

    tk.Button(root, text="Cancel", command=on_cancel).grid(row=3, column=0, pady=10)
    tk.Button(root, text="Done", command=on_done).grid(row=3, column=1, pady=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:20:03 +01:00)


In [12]:
import tkinter as tk

def edit_fields(data, mandatory=(), title="Edit Info"):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    root = tk.Tk()
    root.title(title)

    vars_ = {}
    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=2)
        var = tk.StringVar(value=value)
        vars_[key] = var
        tk.Entry(root, textvariable=var).grid(row=row, column=1, padx=5, pady=2)

    row += 1
    tk.Button(root, text="Cancel", width=10, command=on_cancel).grid(row=row, column=0, pady=10)
    tk.Button(root, text="Done", width=10, command=on_done).grid(row=row, column=1, pady=10)

    root.mainloop()
    return result


time: 16 ms (started: 2025-05-27 23:23:06 +01:00)


In [13]:
bi = {'title': "Once upon a time in America", 
     'author': 'LeLilo, Don.', 
     'year': 2008}
mand = ['title', 'publisher']

time: 0 ns (started: 2025-05-27 23:23:51 +01:00)


In [14]:
edit_fields(bi, mand)

{'title': 'Once upon a time in America',
 'author': 'LeLilo, Don.',
 'year': '2008',
 'publisher': ''}

time: 13.8 s (started: 2025-05-27 23:24:01 +01:00)


In [11]:
edit_book_info()

{'Title': 'The Great Gatsby', 'Author': 'F. Scott Fitzgerald', 'Year': '1925'}

time: 20.4 s (started: 2025-05-27 23:20:11 +01:00)


In [48]:
import tkinter as tk
from tkinter import ttk

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def setup_combobox(box, values):
        def on_key(event):
            val = box.get().lower()
            matches = [v for v in values if val in v.lower()]
            if matches:
                box['values'] = matches
            else:
                box['values'] = values
        box.bind('<KeyRelease>', on_key)
        
    root = tk.Tk()
    root.title(title)
    root.geometry("700x400")  # wider window

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key])
            box.config(width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:39:52 +01:00)


In [71]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")
    
    FONT = ("Segoe UI", 10)

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_next():

    def on_previous()
    
    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=40)
                if match:
                    box.set(match)
        box.bind('<FocusOut>', on_key)
        box.bind('<Tab>', on_key)

    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80, font=FONT)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=82, font=FONT).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Previous", width=12, command=on_previous).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="right", padx=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:54:36 +01:00)


In [76]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process
import pandas as pd

def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
    result = None
    rowcount = len(df)
    fields = df.iloc[i].to_dict()
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def load_row(idx):
        for k in fields:
            val = str(df.iloc[idx].get(k, ""))
            vars_[k].set(val)

    def save_row():
        return {k: vars_[k].get() for k in fields}

    def on_done():
        nonlocal result
        result = (save_row(), current[0])
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_prev():
        if current[0] > 0:
            current[0] -= 1
            load_row(current[0])

    def on_next():
        if current[0] < rowcount - 1:
            current[0] += 1
            load_row(current[0])

    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=70)
                if match and match != current:
                    box.set(match)
                    box.icursor(tk.END)
        box.bind('<KeyRelease>', on_key)

    current = [i]  # mutable index holder
    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, key in enumerate(fields):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar()
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    load_row(i)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Previous", width=12, command=on_prev).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=20)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="left", padx=5)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-28 00:02:34 +01:00)


In [80]:
mandatory = ['type', 'author', 'title', 'year', 'journal', 'publisher']
defaults = {
    k: lib.distinct(k) for k in mandatory
}

time: 32 ms (started: 2025-05-28 00:04:24 +01:00)


In [81]:
# def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
updated = edit_fields_from_df(lib.ref_df.sample(20).reset_index(drop=True)[mandatory], 0, mandatory, title="Edit Book Info", known_values=defaults)
updated

time: 58.5 s (started: 2025-05-28 00:04:29 +01:00)


In [79]:
lib.ref_df.sample(20).reset_index(drop=True).head().T#[mandatory]

,0,1,2,3,4
tag,Peng2019,Cummins2009a,Ali2017,Keich2005,Ridley1976
type,article,article,article,article,book
author,"Peng, Shige","Cummins, J. David and Weiss, Mary A. and Zi, Hongmin","Ali, Muneeb and Shea, Ryan and Nelson, Jude and Freedman, Michael J.","Keich, Uri","Ridley, B. K."
doi,10.1186/s41546-019-0038-2,10.1287/mnsc.45.9.1254,,,10.1119/1.11114
journal,"Probability, Uncertainty and Quantitative Risk",Management Science,,Journal of Computational Biology,
pages,1--8,1254--1269,,416--430,
title,{Law of large numbers and central limit theorem under nonlinear expectations},{Organizational Form and Efficiency: The Coexistence of Stock and Mutual Property-Liability Insu...,{Blockstack: A New Internet for Decentralized Applications},{sFFT: A Faster Accurate Computation of the p-Value of the Entropy Score},"{Time, Space and Things}"
volume,4,45,,12,
year,2019,2009,2017,2005,1976
publisher,,,,,Penguin Books Ltd.


time: 0 ns (started: 2025-05-28 00:04:09 +01:00)


In [72]:
i = i + 1
data = {k:v for k, v in lib.ref_df.loc[i].items() if v != ''}
updated = edit_fields(data, mandatory, title="Edit Book Info", known_values=defaults)
updated

time: 4.7 s (started: 2025-05-27 23:54:36 +01:00)


In [19]:
lib = arcl.Library('uber-library')

time: 31 ms (started: 2025-05-27 23:29:58 +01:00)


In [63]:
i = 202

time: 0 ns (started: 2025-05-27 23:49:21 +01:00)


time: 0 ns (started: 2025-05-27 23:34:26 +01:00)


time: 16 ms (started: 2025-05-27 23:34:28 +01:00)


In [40]:
data = lib.ref_df.querex('select journal, publisher tag ~ Mildenhall2024')[mandatory].iloc[0].to_dict()

time: 15 ms (started: 2025-05-27 23:34:48 +01:00)


{'tag': 'Shen2024',
 'type': 'article',
 'author': 'Shen, Yi and {Van Oosten}, Zachary and Wang, Ruodu',
 'pages': '1--51',
 'title': '{Partially Law-Invariant Risk Measures}',
 'year': '2024',
 'url': 'http://arxiv.org/abs/2401.17265',
 'number': 'June',
 'mendeley-tags': 'WangR',
 'arc-citations': 0,
 'arc-source': 'mendeley'}

time: 0 ns (started: 2025-05-27 23:43:02 +01:00)


In [18]:

data = {"Title": "1984", "Author": "George Orwell", 'Genre': 'Fiction|'}
mandatory = ["Title", "Author", "Year", "Genre"]
known = {"Genre": ["Fiction", "Non-fiction", "Sci-Fi", "Fantasy", "Other"]}
updated = edit_fields(data, mandatory, title="Edit Book Info", known_values=known)
print(updated)


{'Title': '1984', 'Author': 'George Orwell', 'Genre': 'Fiction|', 'Year': ''}
time: 16.3 s (started: 2025-05-27 23:29:16 +01:00)


In [51]:
defaults['type']

['Report',
 'article',
 'book',
 'incollection',
 'inproceedings',
 'manual',
 'misc',
 'phdthesis',
 'techreport',
 'unpublished']

time: 16 ms (started: 2025-05-27 23:41:28 +01:00)


In [200]:
lib._trie = None

time: 0 ns (started: 2025-05-28 10:55:25 +01:00)


In [203]:
lib.to_name_ex('Tsanakas, Andr')

Tsanakas, Andr is not a key...adding


'Tsanakas, Andreas'

time: 15 ms (started: 2025-05-28 10:55:55 +01:00)


In [82]:
import requests

def lookup_doi(doi: str) -> dict | None:
    url = f"https://api.crossref.org/works/{doi}"
    resp = requests.get(url, headers={"User-Agent": "DOI lookup script"})
    if resp.status_code == 200:
        return resp.json()["message"]
    return None


time: 156 ms (started: 2025-05-28 00:06:29 +01:00)


In [83]:
data = lookup_doi("10.1038/nphys1170")
if data:
    print(data["title"][0])
    print(data["author"][0]["family"], data["author"][0]["given"])


Measured measurement
Aspelmeyer Markus
time: 1.11 s (started: 2025-05-28 00:06:42 +01:00)


In [91]:
def extract_bibtex_fields(data: dict) -> dict:
    authors = data.get("author", [])
    author_str = " and ".join(
        f"{a['family']}, {a.get('given', '')}".strip() for a in authors
    )

    return {
        "title": data.get("title", [""])[0],
        "author": author_str,
        "year": data.get("issued", {}).get("date-parts", [[None]])[0][0],
        "journal": data.get("container-title", [""])[0],
        "volume": data.get("volume", ""),
        "number": data.get("issue", ""),
        "pages": data.get("page", ""),
        "doi": data.get("DOI", ""),
        "url": data.get("URL", "")
    }


time: 16 ms (started: 2025-05-28 00:10:55 +01:00)


In [ ]:
for d in dois[40:45]:
    data = lookup_doi(d)
    if data:
        print(d)
        print('-' * len(d))
        print(extract_bibtex_fields(data))
        print()
    else:
        print(f'FAIL {d}\n')

In [95]:
len(dois)

2538

time: 0 ns (started: 2025-05-28 00:12:29 +01:00)


In [84]:
dois = lib.distinct('doi')

time: 0 ns (started: 2025-05-28 00:07:03 +01:00)


In [96]:
def search_doi_by_title(title: str) -> dict | None:
    url = "https://api.crossref.org/works"
    params = {"query.title": title, "rows": 1}
    headers = {
        "User-Agent": "archivum/1.1 (mailto:mynl@me.com)"
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.status_code == 200:
        items = resp.json()["message"]["items"]
        return items[0] if items else None
    return None


time: 0 ns (started: 2025-05-28 00:13:10 +01:00)


In [102]:
result = search_doi_by_title()
if result:
    print(result["DOI"])

10.1007/bf01388809
time: 2.75 s (started: 2025-05-28 00:19:46 +01:00)


In [103]:
extract_bibtex_fields(result)

{'title': 'Heegner points and derivatives ofL-series',
 'author': 'Gross, Benedict H. and Zagier, Don B.',
 'year': 1986,
 'journal': 'Inventiones Mathematicae',
 'volume': '84',
 'number': '2',
 'pages': '225-320',
 'doi': '10.1007/bf01388809',
 'url': 'https://doi.org/10.1007/bf01388809'}

time: 16 ms (started: 2025-05-28 00:19:51 +01:00)


In [100]:
from pprint import pprint

time: 0 ns (started: 2025-05-28 00:18:12 +01:00)


In [ ]:
pprint(result)

In [220]:
lib

Library(uber-library.archivum-config)

time: 0 ns (started: 2025-05-28 11:44:31 +01:00)


In [223]:
len(r)

1

time: 0 ns (started: 2025-05-28 11:45:44 +01:00)


In [ ]:
r = arcc.search(title="Heegner points and derivatives of L-series", rows=1)
arcc.to_bibtex(r[0])

arcc.to_bibtex(r[0], lib)

In [226]:
arcc.to_bibtex(r[0], lib)

{'type': 'article',
 'title': 'Heegner points and derivatives ofL-series',
 'year': 1986,
 'doi': '10.1007/bf01388809',
 'url': 'https://doi.org/10.1007/bf01388809',
 'author': 'Gross, Benedict H. and Zagier, Don B.',
 'journal': 'Inventiones Mathematicae',
 'volume': '84',
 'number': '2',
 'pages': '225-320'}

time: 0 ns (started: 2025-05-28 11:47:01 +01:00)


In [231]:
r = arcr.Reference('Mild2025', 'journal', 'My splendid thing', 2025, author='Mildenhall, Stephen J.')

time: 0 ns (started: 2025-05-28 13:50:42 +01:00)


In [232]:
r

Reference(tag='Mild2025', type='journal', title='My splendid thing', year=2025, author='Mildenhall, Stephen J.', journal=None, volume=None, number=None, month=None, pages=None, edition=None, booktitle=None, editor=None, publisher=None, institution=None, address=None, doi=None, url=None, file=None, source=None, mendeley_tags=None, arc_citations=None, arc_source=None)

time: 0 ns (started: 2025-05-28 13:50:48 +01:00)


In [239]:
r.to_dict(fillna='')

{'tag': 'Mild2025',
 'type': 'journal',
 'title': 'My splendid thing',
 'year': 2025,
 'author': 'Mildenhall, Stephen J.',
 'journal': '',
 'volume': '',
 'number': '',
 'month': '',
 'pages': '',
 'edition': '',
 'booktitle': '',
 'editor': '',
 'publisher': '',
 'institution': '',
 'address': '',
 'doi': '',
 'url': '',
 'file': '',
 'source': '',
 'mendeley_tags': '',
 'arc_citations': '',
 'arc_source': ''}

time: 0 ns (started: 2025-05-28 13:58:26 +01:00)


In [ ]:
r = arcc.search(author='Stephen Mildenhall', rows=5)
print(len(r))

for _ in r:
    ref = arcr.Reference.from_crossref(_, lib)
    pp(ref.to_dict())

In [47]:
lib.distinct('arc-source')

['mendeley']

time: 16 ms (started: 2025-05-28 15:03:19 +01:00)


In [43]:
pp(ref.to_dict())

{'author': 'Mildenhall, Stephen J.',
 'doi': '10.3390/risks5020031',
 'journal': 'Risks',
 'number': '2',
 'pages': '31',
 'source': 'crossref',
 'tag': 'Mildenhall2017',
 'title': 'Actuarial Geometry',
 'type': 'article',
 'url': 'https://doi.org/10.3390/risks5020031',
 'volume': '5',
 'year': 2017}
time: 0 ns (started: 2025-05-28 15:01:57 +01:00)


In [67]:
s = [i for i in lib.distinct('tag') if re.search(r'^AMBest2008', i)][-1]
s

'AMBest2008b'

time: 15 ms (started: 2025-05-28 15:07:13 +01:00)


In [71]:
chr(ord('a') + 1)

'b'

time: 0 ns (started: 2025-05-28 15:07:59 +01:00)


In [72]:
t = s[:-1] + chr(ord(s[-1])+1)
t

'AMBest2008c'

time: 0 ns (started: 2025-05-28 15:08:14 +01:00)
